# "ABC" анализ данных по укрупненному датасету members_13

In [1]:
import numpy as np
import pandas as pd

In [2]:
members_13 = pd.read_csv('members_13.csv', parse_dates = ['birthday', 'entry_date'])
members_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130989 entries, 0 to 130988
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   member_name               130989 non-null  object        
 1   birthday                  130989 non-null  datetime64[ns]
 2   sex                       130989 non-null  object        
 3   profession                130989 non-null  object        
 4   salary_usd$               130989 non-null  int64         
 5   entry_date                130989 non-null  datetime64[ns]
 6   group_id                  130989 non-null  int64         
 7   city_name                 130989 non-null  object        
 8   state_name                130989 non-null  object        
 9   age                       130989 non-null  int64         
 10  fun_club_name             130974 non-null  object        
 11  category_id               130989 non-null  int64         
 12  mo

In [3]:
# все имена членов клубов уникальные
members_13.member_name.nunique()

130989

In [4]:
# все значения в таблице Not NaN
members_13.where(members_13.notna())

,member_name,birthday,sex,profession,salary_usd$,entry_date,group_id,city_name,state_name,age,fun_club_name,category_id,monthly_fee_USD$,category_name,membership_duration_days,"sum_fee_paid, USD$"
0,Vanessa Casey,1960-08-08,F,"Engineer, manufacturing systems",1884,2018-05-24,19927387,Montgomery,West Virginia,63,Events That Make You Smarter,6,7,Education & Learning,1803,421.0
1,Cheryl Carey,1958-12-11,N,Trade union research officer,1518,2019-11-14,19712846,Colonia,New Jersey,65,Mix Professionals Chicago,9,18,Fitness,1264,758.0
2,Kendra Valdez,1962-06-16,M,Press photographer,1669,2018-08-06,10528562,Pinckney,Michigan,61,Madison Square Park Mommies!,25,19,Parents & Family,1729,1095.0
3,Toni Baxter,1992-03-21,F,Emergency planning/management officer,3818,2017-07-18,1291778,Florence,Kentucky,31,San Francisco Dodgeball Meetup Group,32,1,Sports & Recreation,2113,70.0
4,Shaun Zuniga,1999-07-04,N,"Runner, broadcasting/film/video",4962,2017-11-18,26053931,Nickerson,Kansas,24,Passionate Concert Goers,21,9,Music,1990,597.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130984,George Mcdaniel,1943-03-14,M,Comptroller,2393,2018-04-14,1222759,Oakhurst,California,80,RHYTHMflow Explorations,21,21,Music,1843,1290.0
130985,Kevin Diaz DVM,1999-05-16,F,Customer service manager,4874,2018-12-27,2137821,Spring Lake Park,Minnesota,24,Aquatic Voyagers Scuba Club,23,18,Outdoors & Adventure,1586,952.0
130986,Latasha Alvarez MD,1953-07-07,N,Interpreter,2579,2018-09-20,23343110,Newton County,Missouri,70,Chicago Womens Soccer Meet up,32,5,Sports & Recreation,1684,281.0
130987,Hector Davidson,1945-05-05,N,Lexicographer,2890,2018-11-18,10982362,Houston,Alaska,78,Chicago Indonesian (Bahasa Indonesia) Language...,16,23,Language & Ethnic Identity,1625,1246.0


In [5]:
# переименуем "проблемные колонки"
members_13 = members_13.rename(columns={'salary_usd$':'salary', 'monthly_fee_USD$':'monthly_fee', 'sum_fee_paid, USD$':'sum_fee_paid'})

In [6]:
# сгруппируем данные по id клубов, агрегируем по сумме выплаченных взносов,
# ранжируем по убыванию этой суммы
df_abc=members_13.groupby('group_id', as_index=False).agg({'sum_fee_paid':'sum'})\
       .sort_values('sum_fee_paid', ascending=False) 
df_abc

,group_id,sum_fee_paid
6446,23754254,38095.0
4091,19012818,36795.0
2567,10825092,36293.0
710,1288361,36127.0
3425,18383769,34866.0
...,...,...
2328,8156102,0.0
3500,18468547,0.0
2326,8150612,0.0
6270,23348398,0.0


In [7]:
# имеем 329 клубов, в которых видимо членские взносы вообще не взымались.
# это "халявные" клубы и для целей исследования они мало интересны
df_abc['sum_fee_paid'].where(df_abc['sum_fee_paid']==0).value_counts()

0.0    329
Name: sum_fee_paid, dtype: int64

In [8]:
# уберем из дальнейшего исследования эти "халявные" клубы
df_abc=members_13.query('sum_fee_paid !=0')\
        .groupby('group_id', as_index=False).agg({'sum_fee_paid':'sum'})\
        .sort_values('sum_fee_paid', ascending=False) 
df_abc

,group_id,sum_fee_paid
6172,23754254,38095.0
3914,19012818,36795.0
2450,10825092,36293.0
681,1288361,36127.0
3272,18383769,34866.0
...,...,...
6407,24446955,467.0
2261,8552732,445.0
7482,26157417,430.0
6288,24163847,418.0


In [9]:
# создаем переменную общей суммы взносов
total_sum_fees = df_abc.sum_fee_paid.sum()
total_sum_fees

86236301.0

In [10]:
# доля (процент) суммы взносов для каждого клуба в общей сумме взносов
df_abc['share_of_total_fees']=df_abc.sum_fee_paid/total_sum_fees*100
df_abc

,group_id,sum_fee_paid,share_of_total_fees
6172,23754254,38095.0,0.044175
3914,19012818,36795.0,0.042668
2450,10825092,36293.0,0.042086
681,1288361,36127.0,0.041893
3272,18383769,34866.0,0.040431
...,...,...,...
6407,24446955,467.0,0.000542
2261,8552732,445.0,0.000516
7482,26157417,430.0,0.000499
6288,24163847,418.0,0.000485


In [11]:
# добавляем колонку с кумулятивной суммой процентов
df_abc['cum_sum']=df_abc.share_of_total_fees.cumsum()
df_abc

,group_id,sum_fee_paid,share_of_total_fees,cum_sum
6172,23754254,38095.0,0.044175,0.044175
3914,19012818,36795.0,0.042668,0.086843
2450,10825092,36293.0,0.042086,0.128928
681,1288361,36127.0,0.041893,0.170821
3272,18383769,34866.0,0.040431,0.211252
...,...,...,...,...
6407,24446955,467.0,0.000542,99.998105
2261,8552732,445.0,0.000516,99.998621
7482,26157417,430.0,0.000499,99.999120
6288,24163847,418.0,0.000485,99.999605


In [12]:
# присвоим значения  по границам "A<80,B<95,C>95"
df_abc['abc'] = np.where(df_abc['cum_sum']<80, 'A', np.where((df_abc['cum_sum']<95), 'B', 'C'))
df_abc

,group_id,sum_fee_paid,share_of_total_fees,cum_sum,abc
6172,23754254,38095.0,0.044175,0.044175,A
3914,19012818,36795.0,0.042668,0.086843,A
2450,10825092,36293.0,0.042086,0.128928,A
681,1288361,36127.0,0.041893,0.170821,A
3272,18383769,34866.0,0.040431,0.211252,A
...,...,...,...,...,...
6407,24446955,467.0,0.000542,99.998105,C
2261,8552732,445.0,0.000516,99.998621,C
7482,26157417,430.0,0.000499,99.999120,C
6288,24163847,418.0,0.000485,99.999605,C


In [13]:
df_abc.query('abc == "A"')

,group_id,sum_fee_paid,share_of_total_fees,cum_sum,abc
6172,23754254,38095.0,0.044175,0.044175,A
3914,19012818,36795.0,0.042668,0.086843,A
2450,10825092,36293.0,0.042086,0.128928,A
681,1288361,36127.0,0.041893,0.170821,A
3272,18383769,34866.0,0.040431,0.211252,A
...,...,...,...,...,...
4946,20415318,9618.0,0.011153,79.945056,A
2812,14783972,9615.0,0.011150,79.956205,A
7627,26288108,9606.0,0.011139,79.967345,A
7467,26142870,9603.0,0.011136,79.978480,A


In [14]:
# процент клубов класса "А"
4213/7706*100

54.671684401764864

In [15]:
# процент клубов класса "А" от общей суммы взносов
(df_abc.query('abc == "A"').sum_fee_paid.sum())/total_sum_fees*100

79.98960901627726

In [16]:
df_abc.query('abc == "A"').sum_fee_paid.sum()

68980080.0

В нашем случае почти 55% клубов обеспечивают 80% общей суммы взносов.
Принимаем этот факт как данность.
Выведем фрейм со списком наименований клубов класса "А"

In [17]:
df_abc_A = df_abc.query('abc == "A"')['group_id'].to_frame()
df_abc_A

,group_id
6172,23754254
3914,19012818
2450,10825092
681,1288361
3272,18383769
...,...
4946,20415318
2812,14783972
7627,26288108
7467,26142870


In [18]:
# извлечем соответствующие название клубов класса А из таблицы member_13 и проверим по кол-ву уникальных значений
fun_club_class_A=df_abc_A.merge(members_13[['group_id', 'fun_club_name']], how='left', on='group_id')
len(list(fun_club_class_A.fun_club_name.unique()))

4213

In [19]:
# распечатем список клубов класса А. В дальнейшем его можно изпользовать для таргетированной рекламы.
array = fun_club_class_A.fun_club_name.unique()
class_A_clubs = pd.DataFrame(array, columns = ['club_name'])
class_A_clubs

,club_name
0,Cultural Explorations NYC
1,Films@Facets
2,New York Kayak Polo
3,The New York City MBTI Meetup Group
4,Intro to Voice Training -- NYC
...,...
4208,UES New York Book Club Meetup
4209,Composing Film/Game Music SF
4210,Manchester United NYC watch parties!
4211,New York Adult Kick Scooter Meetup


# "XYZ" анализ

In [41]:
# сгруппируем данные по group_id клубов и аггрегируем по числу членов в каждом
df_xyz = members_13.groupby('group_id', as_index = False).agg({'member_name': 'count'}).sort_values('member_name', ascending = False)
df_xyz

,group_id,member_name
2896,14213992,31
3247,17901622,31
6446,23754254,31
6578,24189636,30
2442,9496682,30
...,...,...
7517,25870263,5
2394,8798982,5
5091,20339652,5
3228,17759102,5


In [42]:
df_xyz.describe([0.33, 0.66, 0.99])[["member_name"]].T

,count,mean,std,min,33%,50%,66%,99%,max
member_name,8035.0,16.302302,3.994437,4.0,14.0,16.0,18.0,26.0,31.0


In [43]:
# присвоим значения  по границам Z=33%, Y=66%
df_xyz['xyz'] = np.where(df_xyz['member_name']<14, 'Z', np.where((df_xyz['member_name']<18), 'Y', 'X'))
df_xyz

,group_id,member_name,xyz
2896,14213992,31,X
3247,17901622,31,X
6446,23754254,31,X
6578,24189636,30,X
2442,9496682,30,X
...,...,...,...
7517,25870263,5,Z
2394,8798982,5,Z
5091,20339652,5,Z
3228,17759102,5,Z


In [44]:
df_xyz.query('xyz == "X"')

,group_id,member_name,xyz
2896,14213992,31,X
3247,17901622,31,X
6446,23754254,31,X
6578,24189636,30,X
2442,9496682,30,X
...,...,...,...
1195,1673270,18,X
467,594908,18,X
2291,7859162,18,X
459,565564,18,X


In [45]:
round((2946/8035*100),2)

36.66

In [52]:
# объединим таблицы ABC и XYZ групп клубов
df_total = df_abc.merge(df_xyz, on ='group_id')
df_total

,group_id,sum_fee_paid,share_of_total_fees,cum_sum,abc,member_name,xyz
0,23754254,38095.0,0.044175,0.044175,A,31,X
1,19012818,36795.0,0.042668,0.086843,A,28,X
2,10825092,36293.0,0.042086,0.128928,A,29,X
3,1288361,36127.0,0.041893,0.170821,A,28,X
4,18383769,34866.0,0.040431,0.211252,A,28,X
...,...,...,...,...,...,...,...
7701,24446955,467.0,0.000542,99.998105,C,9,Z
7702,8552732,445.0,0.000516,99.998621,C,8,Z
7703,26157417,430.0,0.000499,99.999120,C,8,Z
7704,24163847,418.0,0.000485,99.999605,C,8,Z


In [53]:
pd.crosstab(df_total.abc, df_total.xyz)

xyz,X,Y,Z
abc,,,
A,1876,1633,704
B,495,676,639
C,454,637,592


In [54]:
print('зеленая группа', round((((1876+495+454+1633)/7706)*100), 2),'%')

зеленая группа 57.85 %


In [58]:
df_total_AX = df_total.loc[(df_total.abc=='A') & (df_total.xyz=='X')]
df_total_AX 

,group_id,sum_fee_paid,share_of_total_fees,cum_sum,abc,member_name,xyz
0,23754254,38095.0,0.044175,0.044175,A,31,X
1,19012818,36795.0,0.042668,0.086843,A,28,X
2,10825092,36293.0,0.042086,0.128928,A,29,X
3,1288361,36127.0,0.041893,0.170821,A,28,X
4,18383769,34866.0,0.040431,0.211252,A,28,X
...,...,...,...,...,...,...,...
4193,4021122,9683.0,0.011228,79.777349,A,19,X
4197,24195835,9666.0,0.011209,79.822215,A,18,X
4199,20390990,9645.0,0.011184,79.844584,A,22,X
4204,21188207,9625.0,0.011161,79.900432,A,19,X


In [59]:
# извлечем соответствующие название клубов класса АX из таблицы member_13 и проверим по кол-ву уникальных значений
fun_club_class_AX=df_total_AX.merge(members_13[['group_id', 'fun_club_name']], how='left', on='group_id')
len(list(fun_club_class_AX.fun_club_name.unique()))

1876

In [60]:
# распечатем список клубов класса АX. В дальнейшем его можно изпользовать для таргетированной рекламы.
array = fun_club_class_AX.fun_club_name.unique()
class_AX_clubs = pd.DataFrame(array, columns = ['club_name'])
class_AX_clubs

,club_name
0,Cultural Explorations NYC
1,Films@Facets
2,New York Kayak Polo
3,The New York City MBTI Meetup Group
4,Intro to Voice Training -- NYC
...,...
1871,New York Mets Meetup Group
1872,San Francisco Free Art and Music Events
1873,The Soca Dance Experience
1874,Love My Body Again San Francisco Meetup
